### Coreference Resolution & NER
* Spacy itself (en_coreference_web_trf)

In [ ]:
# my environment (fc)

# Python == 3.9.19
# spacy == 3.7.6 (Latest)
# spacy-transformers == 1.3.5

In [ ]:
!pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.0/en_coreference_web_trf-3.4.0a0-py3-none-any.whl#egg=en_coreference_web_trf

In [1]:
import spacy
# import neuralcoref
from spacy import displacy
from wasabi import msg

In [2]:
nlp = spacy.load("en_coreference_web_trf")

/Users/minjoo/opt/anaconda3/envs/fastcoref2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Input text for coreference resolution
text3 = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."

In [4]:
# Process the text with SpaCy
doc = nlp(text3)

In [5]:
# Print out clusters -> Coreference Resolution OK
msg.info("Found clusters")
for cluster in doc.spans:
    print(f"{cluster}: {doc.spans[cluster]}")

ℹ Found clusters
coref_clusters_1: [Google in, the company took]
coref_clusters_2: [Sebastian Thrun started, him seriously]


In [6]:
# since Spacy doesn't have direct coreference resolver
# we define lightweight function for resolving references in text
def resolve_references(doc) -> str:
    """
    Function for resolving references with the coref ouput
    doc (Doc): The Doc object processed by the coref pipeline
    RETURNS (str): The Doc string with resolved references
    """
    # token.idx : token.text
    token_mention_mapper = {}
    output_string = ""
    clusters = [
    val for key, val in doc.spans.items() if key.startswith("coref_cluster")
        ]

    # Iterate through every found cluster
    for cluster in clusters:
        first_mention = cluster[0]
        # Iterate through every other span in the cluster
        for mention_span in list(cluster)[1:]:
            # Set first_mention as value for the first token in mention_span in the token_mention_mapper
            token_mention_mapper[mention_span[0].idx] = first_mention.text + mention_span[0].whitespace_
            for token in mention_span[1:]:
                # Set empty string for all the other tokens in mention_span
                token_mention_mapper[token.idx] = ""
    # Iterate through every token in the Doc
    for token in doc:
        # Check if token exists in token_mention_mapper
        if token.idx in token_mention_mapper:
            output_string += token_mention_mapper[token.idx]
        # Else add original token text
        else:
            output_string += token.text + token.whitespace_
    return output_string

In [9]:
msg.info("Document with resolved references --> Looks Great!")
print(resolve_references(doc))

ℹ Document with resolved references --> Looks Great!
When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of Google in Sebastian Thrun started .


In [10]:
# Print out component names -> since there's no NER tagger, we can't directly use this for Named Entity Recognition.

msg.info("Pipeline components")
for i, pipe in enumerate(nlp.pipe_names):
    print(f"{i}: {pipe}")

ℹ Pipeline components
0: sentencizer
1: transformer
2: coref
3: span_resolver
4: span_cleaner


In [17]:
!python -m spacy download en_core_web_sm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 4.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [18]:
test_model = spacy.load('en_core_web_sm')

In [23]:
msg.info("if we check the components from the pipeline, we can see there's no transformer and coref in the basic model")
for i, pipe in enumerate(test_model.pipe_names):
    print(f"{i}: {pipe}")

ℹ if we check the components from the pipeline, we can see there's no
transformer and coref in the basic model
0: tok2vec
1: tagger
2: parser
3: attribute_ruler
4: lemmatizer
5: ner


In [14]:
# So we are going to combine two different models

# This base model (nlp) can be anything from Spacy
!python -m spacy download en_core_web_md
# nlp = spacy.load("en_core_web_trf")
nlp = spacy.load("en_core_web_md")
# nlp = spacy.load("en_core_web_sm")

nlp_coref = spacy.load("en_coreference_web_trf")

# use replace_listeners for the coref components
nlp_coref.replace_listeners("transformer", "coref", ["model.tok2vec"])
nlp_coref.replace_listeners("transformer", "span_resolver", ["model.tok2vec"])

# we won't copy over the span cleaner
nlp.add_pipe("coref", source=nlp_coref)
nlp.add_pipe("span_resolver", source=nlp_coref)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 7.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


/Users/minjoo/opt/anaconda3/envs/fastcoref2/lib/python3.9/site-packages/spacy/language.py:716: UserWarning: [W113] Sourced component 'coref' may not work as expected: source vectors are not identical to current pipeline vectors.
  warnings.warn(Warnings.W113.format(name=source_name))
/Users/minjoo/opt/anaconda3/envs/fastcoref2/lib/python3.9/site-packages/spacy/language.py:716: UserWarning: [W113] Sourced component 'span_resolver' may not work as expected: source vectors are not identical to current pipeline vectors.
  warnings.warn(Warnings.W113.format(name=source_name))


In [28]:
doc2= nlp(text3)

In [29]:
# Print out component names
# Now we have NER on the 5th!

msg.info("Pipeline components : Now we have NER on the 5th!")
for i, pipe in enumerate(nlp.pipe_names):
    print(f"{i}: {pipe}")

ℹ Pipeline components : Now we have NER on the 5th!
0: tok2vec
1: tagger
2: parser
3: attribute_ruler
4: lemmatizer
5: ner
6: coref
7: span_resolver


In [30]:
msg.info("Document with resolved references --> Still looking good with the new model")
print(resolve_references(doc2))

ℹ Document with resolved references --> Still looking good with the new
model
When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of Google in Sebastian Thrun started .


In [31]:
resolved_doc = nlp(resolve_references(doc2))

In [33]:
msg.info("NER looking good too!")
displacy.render(resolved_doc, style="ent")

ℹ NER looking good too!
